In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [3]:
df=pd.read_csv("/content/drive/My Drive/CASE Studies/CASE1/dataset.csv")
df.columns

Index(['Price', 'Airline', 'Cabin', 'Dept_city', 'Dept_date', 'Dept_month',
       'arrival_city', 'stops', 'layover_cities', 'Departure_Date',
       'Arrival_Date', 'duration', 'weekday', 'day_name', 'dept_hours',
       'arrival_hours', 'Dept_flights_time', 'arrival_flights_time'],
      dtype='object')

In [4]:
df["optimal_time"]=df["dept_hours"]-4

In [5]:
for i in range(df.shape[0]):
  if df["optimal_time"].iloc[i]<0:
    df["optimal_time"].iloc[i] +=24

df["optimal_time"].describe()


/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


count    315985.000000
mean          8.914271
std           5.148739
min           0.000000
25%           4.000000
50%           8.000000
75%          13.000000
max          23.000000
Name: optimal_time, dtype: float64

In [6]:
df.drop(columns=["layover_cities","Departure_Date","Arrival_Date","day_name","arrival_flights_time","arrival_hours"],inplace=True,axis=1)

In [7]:
ind=[]
for i in range(df.shape[0]):
  if "," in df["Airline"].iloc[i]:
    ind.append(i)
df.drop(ind,inplace=True,axis=0)

In [8]:
air=df["Airline"].value_counts().sort_values(ascending=False)
air
l1 = []
l1.append(df[df["Airline"] == "flybig"].index)
l1.append(df[df["Airline"] == "TruJet"].index)
l1.append(df[df["Airline"] == "Air India Express"].index)
#print(l1)
l2 = []
for i in l1:
    for j in i:
        l2.append(j)
df.drop(l2,inplace=True,axis=0)

In [9]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 288090 entries, 0 to 315984
Data columns (total 13 columns):
 #   Column             Non-Null Count   Dtype  
---  ------             --------------   -----  
 0   Price              288090 non-null  int64  
 1   Airline            288090 non-null  object 
 2   Cabin              288090 non-null  object 
 3   Dept_city          288090 non-null  object 
 4   Dept_date          288090 non-null  int64  
 5   Dept_month         288090 non-null  object 
 6   arrival_city       288090 non-null  object 
 7   stops              288090 non-null  int64  
 8   duration           288090 non-null  float64
 9   weekday            288090 non-null  int64  
 10  dept_hours         288090 non-null  int64  
 11  Dept_flights_time  288090 non-null  object 
 12  optimal_time       288090 non-null  int64  
dtypes: float64(1), int64(6), object(6)
memory usage: 30.8+ MB


In [10]:
dff=df.copy()

In [11]:
x=["Airline","Cabin","Dept_month","Dept_flights_time","Dept_city","arrival_city"]
from sklearn.preprocessing import LabelEncoder
for i in x:
  encoder=LabelEncoder()
  df[i]=encoder.fit_transform(df[[i]])

/usr/local/lib/python3.7/dist-packages/sklearn/preprocessing/_label.py:251: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [12]:
x=df.drop(["Price"],axis=1)
y=df["Price"]


In [13]:
import xgboost as xg
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.2,random_state=0)
x_tr,x_cv,y_tr,y_cv=train_test_split(x_train,y_train,test_size=0.2,random_state=0)

In [17]:
from sklearn.ensemble import RandomForestRegressor
rfg=RandomForestRegressor(n_estimators=100 )
rfg.fit(x_tr,y_tr)
pred = rfg.predict(x_cv)
rmse = np.sqrt(MSE(y_cv, pred))
r2= r2_score(y_cv, pred)
print("RMSE : % f" %(rmse))
print("R2 : % f" %(r2))

RMSE :  3329.850088
R2 :  0.971854


In [18]:
a=df["Cabin"].value_counts().keys()
b=dff["Cabin"].value_counts().keys()
cabin=dict()
for i in range(len(a)):
  cabin[b[i]]=a[i]

In [19]:
airline={3:'indigo',
         0:'airindia',
         5:'vistara',
         2:'goair',
         4:'spicehjet',
         1:'airasia'}

In [20]:
a=df["Dept_city"].value_counts().keys()
b=dff["Dept_city"].value_counts().keys()
D_city=dict()
for i in range(len(a)):
  D_city[b[i]]=a[i]

In [21]:
a=df["arrival_city"].value_counts().keys()
b=dff["arrival_city"].value_counts().keys()
A_city=dict()
for i in range(len(a)):
  A_city[b[i]]=a[i]

In [22]:
dcity=input("Enter the Departure ciity : ")
acity=input("Enter the Arrival ciity: ")
date=input("Enter the date of of the journey (YYYY-MM_DD): ")
time=input("Enter the time u want to fly: ")
cab=input("Enter the Cabin: ")

Enter the Departure ciity : New Delhi
Enter the Arrival ciity: Mumbai 
Enter the date of of the journey (YYYY-MM_DD): 2021-05-10
Enter the time u want to fly: 5
Enter the Cabin: E


In [23]:
d=date+' 0'+time+':00:00'
month=pd.to_datetime(d).month
hour=pd.to_datetime(d).hour
day=pd.to_datetime(d).day
weekday=pd.to_datetime(d).dayofweek
optimal_hour=hour-4

In [24]:
for i in range(optimal_hour-1,optimal_hour+3):
  for j in range(5):
    p=rfg.predict([[ j ,1 ,month,2 ,i ,260, day ,A_city[acity] ,D_city[dcity] ,cabin[cab] , weekday ,hour]])
    print(f"airline={airline[j]} with stops=1, destination_city={dcity},arrivalcity={acity} , time to depart={hour} ,optimaltimehrs={i} Price={p[0]}")
    print("**"*100) 

airline=airindia with stops=1, destination_city=New Delhi,arrivalcity=Mumbai  , time to depart=5 ,optimaltimehrs=0 Price=13049.8
********************************************************************************************************************************************************************************************************
airline=airasia with stops=1, destination_city=New Delhi,arrivalcity=Mumbai  , time to depart=5 ,optimaltimehrs=0 Price=4476.4
********************************************************************************************************************************************************************************************************
airline=goair with stops=1, destination_city=New Delhi,arrivalcity=Mumbai  , time to depart=5 ,optimaltimehrs=0 Price=4468.52
********************************************************************************************************************************************************************************************************
airline=indigo 